# Building An Effectful Fragment, Modularly

## Setting things up

* First we need to bring the monad instances into scope
* These instances contain the definitions of `return` and `bind` for the various notions of "effect" that we'll be considering

In [158]:
{-# LANGUAGE FlexibleInstances, FlexibleContexts #-}

import Prelude hiding (log)
import Control.Monad.Identity
import Control.Monad.List
import Control.Monad.Reader
import Control.Monad.Writer
import Control.Monad.State
import Control.Monad.RWS
import Control.Monad.Cont
:load GrammarUtils

* Then we need to define the types we'll need for denotations, and set up a model
  in which to interpret those types

* First up, we declare a type `E` and populate it with two individuals, `John` and
  `Mary`. Right now, these objects are just dots in the Haskell universe. The only
  thing we know about them is that `John == John` and `Mary == Mary`.

In [160]:
data E = John | Mary deriving (Eq, Show)

* Next we define a type for Booleans, which, fortunately Haskell already knows about.
* Haskell also already knows about functional types, like `E -> T`. Assuming we restrict
  ourselves to the exponential closure of `E` and `T`, we've embedded the usual simple
  theory of types.
* And finally to get things going, we'll add a single relation to the model. Everybody
  likes Mary; nobody likes John.

In [161]:
type T = Bool

likes' :: E -> E -> T
likes' x _ = x == Mary

* And that's it for the model theory. Onto the language!

## Lexicon

* The lexicon is basically trivial: names and predicates are directly referential
  ("john" means `John`; "likes" means `likes'`, etc.)

* But with one twist: we inject these simple values into an effect-less structure

* We do this in anticipation of the fact that these pieces might eventually compose
  with effect-**ful** language, and also because later we might want to imbue these
  words with effects themselves

In [162]:
john, mary :: Monad m => m E
john = return John
mary = return Mary

likes :: Monad m => m (E -> E -> Bool)
likes = return likes'

* The cool thing about this is that because *every* monad defines its own well-behaved
  notion of a "pure" computation (its own `return`), we don't have to specify which
  monadic type these words denote in
  
* We can specify it later at any point, and the Haskell type-inference engine will
  resolve the ambiguous computation according to the relevant definition:

In [163]:
print (john :: Identity E)
print (john :: Maybe E)
print (john :: [E])

Identity John

Just John

[John]

* For convenience, we can also exploit Haskell's family of "run" commands,
  which are basically identity functions with specialized types

In [164]:
:t runIdentity
:t runReader
:t runList

runIdentity :: forall a. Identity a -> a

runReader :: forall r a. Reader r a -> r -> a

runList :: forall a. [a] -> [a]

In [165]:
print (runIdentity john)
print (runMaybe john)
print (runList john)

John

Just John

[John]

## Grammar: Monadic Application

* Here we lift forward and backward function application through an arbitrary monad

* Remember, every instance of a monad will provide law-abiding definitions for the
  functions `return` and `(>>=)`

* So what's going on here is that we don't directly apply functions to their arguments,
  because one or both of those things could have side effects, which need to be dealt
  with
  
* Instead, we use the monadic operations to extract the values of the two constituents
  before applying one to the other, in the process *sequencing* their effects from left
  to right

In [166]:
-- Forward function application
(</>) :: Monad m => m (a -> b) -> m a -> m b
mf </> mx = mf >>= \f ->
              mx >>= \x ->
                return (f x)
                
-- Backward function application     
(<\>) :: Monad m => m a -> m (a -> b) -> m b
mx <\> mf = mx >>= \x ->
              mf >>= \f ->
                return (f x)

## Making Sentences

* With these slashes as our "modes of composition", we build the meanings of sentences
  up in the usual type-driven way; that is, we choose the slash that makes the type fit

In [175]:
sen1 = john <\> (likes </> mary)
sen2 = mary <\> (likes </> john)

* Notice that Haskell is actually inferring the types of the larger constituents,
  based on the types of the parts and the mode of composition

In [176]:
:type sen1
:type sen2

sen1 :: forall (m :: * -> *). Monad m => m Bool

sen2 :: forall (m :: * -> *). Monad m => m Bool

* So at this point `sen1` and `sen2` are monad-neutral

* They represent computations that first extract a value from the denotation of their
  subject, then a value from that of the predicate, and then one from the object, in
  that order, accruing side effects as they go
  
* With those values in hand, it uses the direction of the slashes (really, the
  types) to apply the pieces to one another
  
* To inspect the result, we need only specify what monad we want to see it in

In [171]:
runIdentity sen1
runIdentity sen2

True

False

## Adding An Effect: Writer

In [120]:
log :: (Show a, MonadWriter String m) => m a -> m a
log m = m >>= (\x -> writer (x, "Log " ++ show x ++ ". "))

-- sen2 :: MonadWriter String m => m Bool
sen2 = log mary <\> (likes </> log john)

In [121]:
runWriter sen2

(False,"Log Mary. Log John. ")

In [122]:
someone :: MonadPlus m => m E
someone = john `mplus` mary

sen3 :: MonadPlus m => m Bool
sen3 = someone <\> (likes </> john)

In [123]:
runListT sen3

[False,False]

In [124]:
sen4 :: (MonadPlus m, MonadWriter String m) => m Bool
sen4 = log someone <\> (likes </> john)

In [125]:
runListT (runWriterT sen4)

[(False,"Log John. "),(False,"Log Mary. ")]

In [126]:
data Context = Context {speaker :: E, time :: Int}

thisContext :: Context
thisContext = Context {speaker = John, time = 0}

In [127]:
me :: MonadReader Context m => m E
me = asks speaker

sen5 :: MonadReader Context m => m Bool
sen5 = mary <\> (likes </> me)

In [128]:
runReader sen5 thisContext
-- thisContext = {speaker = John, time = 0}

False

In [129]:
type Stack = [E]

discourseInitial :: Stack
discourseInitial = []

justMentionedJohn :: Stack
justMentionedJohn = [John]

In [130]:
pro :: MonadState Stack m => m E
pro = gets head

sen6 :: MonadState Stack m => m Bool
sen6 = mary <\> (likes </> pro)

In [131]:
runState sen6 discourseInitial
-- discourseInitial = []

Prelude.head: empty list

In [132]:
runState sen6 justMentionedJohn
-- justMentionedJohn = [John]

(False,[John])

In [133]:
sen65 :: (MonadState Stack m, MonadWriter String m) => m Bool
sen65 = mary <\> (likes </> log pro)

In [134]:
runWriter (runStateT sen65 justMentionedJohn)

((False,[John]),"Log John. ")

In [135]:
push :: MonadState Stack m => m E -> m E
push m = m >>= \x -> state (\s -> (x, x:s))

sen7 :: MonadState Stack m => m Bool
sen7 = push mary <\> (likes </> pro)

In [136]:
:type state

state :: forall s (m :: * -> *) a. MonadState s m => (s -> (a, s)) -> m a

In [137]:
runState sen7 justMentionedJohn

(True,[Mary,John])

In [138]:
sen8 :: (MonadPlus m, MonadState Stack m) => m Bool
sen8 = push someone <\> (likes </> pro)

In [139]:
runListT (runStateT sen8 [])

[(False,[John]),(True,[Mary])]

In [140]:
sen9 :: (MonadPlus m, MonadState Stack m, MonadWriter String m) => m Bool
sen9 = push someone <\> (likes </> log pro)

In [141]:
runListT (runWriterT (runStateT sen9 []))
runListT (runStateT (runWriterT sen9) [])
runListT (runRWST sen9 thisContext [])

[((False,[John]),"Log John. "),((True,[Mary]),"Log Mary. ")]

[((False,"Log John. "),[John]),((True,"Log Mary. "),[Mary])]

[(False,[John],"Log John. "),(True,[Mary],"Log Mary. ")]

In [142]:
sen10 :: (MonadPlus m, MonadRWS Context String Stack m) => m Bool
sen10 = push someone <\> (likes </> log me)

In [143]:
runListT (runRWST sen10 thisContext discourseInitial)

[(False,[John],"Log John. "),(False,[Mary],"Log John. ")]

In [144]:
everyone :: Monad m => ContT Bool m E
everyone = ContT $ \k -> k John `andM` k Mary
  where andM = liftM2 (&&)

lower :: Monad m => ContT a m a -> m a
lower t = runContT t return

In [145]:
sen11 :: Monad m => ContT Bool m Bool
sen11 = mary <\> (likes </> everyone)

In [146]:
runIdentity (lower sen11)

False

In [147]:
sen12 :: MonadState Stack m => ContT Bool m Bool
sen12 =  mary <\> (likes </> push everyone)

In [148]:
runState (lower sen12) discourseInitial

(False,[Mary,John])

In [149]:
sen13 :: (MonadPlus m, MonadState Stack m, MonadWriter String m) => ContT Bool m Bool
sen13 = log someone <\> (likes </> push everyone)

In [150]:
runListT (runWriterT (runStateT (lower sen13) discourseInitial))

[((False,[Mary,John]),"Log John. "),((False,[Mary,John]),"Log Mary. ")]